<center><h1>Paper results - (1º Notebook)</h1></center>

##### Experiments:

1. Comparison between global vs regional using linear classifier (LSC-LBF)
 
##### Output:
 - **Paper (Table 3):** Performance metrics of all data sets during G/R-LSC experiment.

 - **Paper (Figure 5):** Train and test accuracy distributions in G/R-LSC performance comparison.
 - **Paper (Figure 6):** Distribution of $K_{opt}$ in R-LSC

In [67]:
import warnings
warnings.filterwarnings('ignore')

In [68]:
%%time

import numpy as np 
import pandas as pd 

from math import ceil
from load_dataset import datasets

from devcode.utils import dummie2multilabel, scale_feat
from devcode.models.som import SOM
from devcode.models.regional_learning import RegionalModel
from devcode import run_round

from devcode.utils.metrics import DB

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.cluster import KMeans

CPU times: total: 0 ns
Wall time: 0 ns


In [69]:
test_size     = 0.2
scaleType     = 'min-max'
n_resamplings = 100

num = 3

alphas = [0.1, 0.3, 0.5]     #np.linspace(0.1, 0.5,  num=num).tolist()
sigmas = [3.0, 6.5, 10.0]    # np.linspace(3,    10,   num=num).tolist()
epochs = [100]               # np.linspace(100,  500, num=num, dtype='int').tolist()

# vector of random states for train/test split
random_states = np.random.randint(np.iinfo(np.int32).max, size=n_resamplings).tolist()
cases = [
    {
         "dataset_name" : dataset_name
        ,"random_state":  random_state
        ,"som_params":    { "alpha0"  : alpha0
                           ,"sigma0"  : sigma0
                           ,"nEpochs" : nEpochs
                          }
    }
    # hyperparameters possible values
    for dataset_name in datasets.keys()
    for random_state in random_states
    for alpha0       in alphas
    for sigma0       in sigmas
    for nEpochs      in epochs
]

In [70]:
som_params = [
    {
        "alpha0": alpha0
        , "sigma0": sigma0
        , "nEpochs": nEpochs
    }
    for alpha0 in alphas
    for sigma0 in sigmas
    for nEpochs in epochs
]

### 1. Regional linear classifier (LSC-LBF)

In [71]:
# loading simulation results
df_results = pd.read_csv('results/regional-results/ROLS - all - n_res=100 - 2019-07-10.csv')
df_results.head()

,dataset_name,random_state,alpha0,sigma0,nEpochs,cm_tr,cm_ts
0,vc2c,127815836,0.1,3.0,100,[156 13 15 64],[32 9 7 14]
1,vc2c,127815836,0.1,10.0,100,[153 16 16 63],[32 9 6 15]
2,vc2c,127815836,0.1,6.5,100,[162 7 21 58],[34 7 9 12]
3,vc2c,127815836,0.1,10.0,300,[157 12 21 58],[32 9 8 13]
4,vc2c,127815836,0.1,6.5,300,[155 14 19 60],[33 8 7 14]


In [72]:
from devcode.utils.evaluation import cm2acc

header = list(som_params[0].keys()) + ['Minimum', 'Maximum', 'Median', 'Mean', 'Std. Deviation']

df_ds = {}
for dataset_name in datasets:  # For this specific dataset
    print(dataset_name)
    df = df_results.loc[df_results['dataset_name'] == dataset_name]  # get simulation results

    df['alpha0'] = df['alpha0'].round(decimals=2)
    df['sigma0'] = df['sigma0'].round(decimals=2)

    count = 0
    df_data = np.empty((len(som_params), len(header)))  # matriz que guardará resultados numéricos
    for params in som_params:
        params['alpha0'] = round(params['alpha0'], 2)

        df_case = df.loc[(df['alpha0'] == params['alpha0']) &
                         (df['sigma0'] == params['sigma0']) &
                         (df['nEpochs'] == params['nEpochs'])]

        # converting confusion matrix from string to numpy array
        # cm_ts = np.array([[int(x) for x in result[1:-1].split()] for result in df_case['cm_ts'].values])

        cm = df_case['cm_ts'].values
        n_runs = cm.shape[0]

        rows = [np.fromstring(df_case['cm_ts'].values[i][1:-1], sep=" ") for i in range(n_runs)]
        cm_ts = np.array(rows)

        # data = cm_ts
        length = cm_ts.shape[1]
        cm_side = int(np.sqrt(length))

        #display(cm_ts)
        
        acc = [0] * len(cm_ts)
        for i in range(len(cm_ts)):
            cm = np.reshape(cm_ts[i], (cm_side, cm_side))
            #display(cm)
            acc[i] = np.trace(cm) / np.sum(cm)
            
        hyperparams = [params['alpha0'], params['sigma0'], params['nEpochs']]
        mean_stats  = [min(acc), max(acc), np.median(acc), np.mean(acc), np.std(acc)]
            
        metrics = hyperparams + mean_stats

        df_data[count, :] = np.matrix(metrics)
        count += 1

    df_ds[dataset_name] = pd.DataFrame(df_data, columns=header)
    display(df_ds[dataset_name])
    print('-' * 100, '\n' * 2)

vc2c


,alpha0,sigma0,nEpochs,Minimum,Maximum,Median,Mean,Std. Deviation
0,0.1,3.0,100.0,0.677419,0.919355,0.838710,0.827097,0.050134
1,0.1,6.5,100.0,0.693548,0.903226,0.838710,0.828871,0.046573
2,0.1,10.0,100.0,0.725806,0.951613,0.838710,0.832419,0.045040
3,0.3,3.0,100.0,0.709677,0.935484,0.830645,0.829839,0.045240
4,0.3,6.5,100.0,0.725806,0.935484,0.838710,0.832097,0.047470
5,0.3,10.0,100.0,0.709677,0.935484,0.822581,0.828710,0.045206
6,0.5,3.0,100.0,0.693548,0.919355,0.838710,0.828387,0.045191
7,0.5,6.5,100.0,0.725806,0.919355,0.830645,0.830000,0.041899
8,0.5,10.0,100.0,0.693548,0.935484,0.838710,0.830323,0.049476


---------------------------------------------------------------------------------------------------- 


vc3c


,alpha0,sigma0,nEpochs,Minimum,Maximum,Median,Mean,Std. Deviation
0,0.1,3.0,100.0,0.645161,0.919355,0.822581,0.811129,0.049624
1,0.1,6.5,100.0,0.693548,0.919355,0.814516,0.810806,0.044748
2,0.1,10.0,100.0,0.709677,0.903226,0.822581,0.814032,0.042334
3,0.3,3.0,100.0,0.693548,0.919355,0.822581,0.815645,0.043832
4,0.3,6.5,100.0,0.693548,0.903226,0.822581,0.812742,0.048650
5,0.3,10.0,100.0,0.677419,0.919355,0.822581,0.816452,0.048750
6,0.5,3.0,100.0,0.693548,0.919355,0.822581,0.813871,0.050822
7,0.5,6.5,100.0,0.677419,0.903226,0.822581,0.810806,0.048001
8,0.5,10.0,100.0,0.645161,0.903226,0.806452,0.808548,0.049327


---------------------------------------------------------------------------------------------------- 


wf24f


...

---------------------------------------------------------------------------------------------------- 


wf2f


,alpha0,sigma0,nEpochs,Minimum,Maximum,Median,Mean,Std. Deviation
0,0.1,3.0,100.0,0.769231,0.960623,0.908883,0.911016,0.023783
1,0.1,6.5,100.0,0.855311,0.957875,0.907509,0.907326,0.017636
2,0.1,10.0,100.0,0.862637,0.959707,0.908883,0.910046,0.015775
3,0.3,3.0,100.0,0.804029,0.964286,0.907509,0.906364,0.020521
4,0.3,6.5,100.0,0.847985,0.958791,0.905678,0.905696,0.016253
5,0.3,10.0,100.0,0.830586,0.962454,0.907051,0.905476,0.018077
6,0.5,3.0,100.0,0.798535,0.961538,0.902930,0.897821,0.026447
7,0.5,6.5,100.0,0.813187,0.961538,0.903846,0.904396,0.018936
8,0.5,10.0,100.0,0.800366,0.966117,0.902930,0.899945,0.024333


---------------------------------------------------------------------------------------------------- 


pk


,alpha0,sigma0,nEpochs,Minimum,Maximum,Median,Mean,Std. Deviation
0,0.1,3.0,100.0,0.717949,1.000000,0.871795,0.860513,0.059177
1,0.1,6.5,100.0,0.692308,1.000000,0.871795,0.865128,0.062030
2,0.1,10.0,100.0,0.666667,1.000000,0.871795,0.863077,0.057015
3,0.3,3.0,100.0,0.692308,0.974359,0.871795,0.864359,0.059617
4,0.3,6.5,100.0,0.717949,0.974359,0.871795,0.864103,0.057507
5,0.3,10.0,100.0,0.692308,0.974359,0.871795,0.868974,0.055576
6,0.5,3.0,100.0,0.666667,0.974359,0.871795,0.860769,0.059833
7,0.5,6.5,100.0,0.692308,0.974359,0.871795,0.862308,0.050592
8,0.5,10.0,100.0,0.743590,1.000000,0.871795,0.868205,0.058473


---------------------------------------------------------------------------------------------------- 




### 2. Global linear classifier (LSC-LBF)

In [73]:
# loading simulation results
df_results = pd.read_csv('results/local-results/GOLS - n_init 100 - 2019-12-11.csv')
df_results.head()

,Unnamed: 0,vc2c,vc3c,wf24f,wf4f,wf2f,pk
0,0,0.870968,0.741935,0.647436,0.737179,0.702381,0.923077
1,1,0.774194,0.741935,0.663004,0.727106,0.720696,0.871795
2,2,0.854839,0.790323,0.629121,0.696886,0.724359,0.897436
3,3,0.838710,0.951613,0.632784,0.709707,0.717033,0.794872
4,4,0.903226,0.758065,0.635531,0.744505,0.728022,0.948718
